In [ ]:
from numpy import *
from scipy.interpolate import griddata
from matplotlib import cm
import matplotlib.pyplot as plt

In [ ]:
from module_poisson_2D import*

In [ ]:
# Domain of integration, look into func_2D module to add your own problem or initial condition

def domain(icase):
    
    
    if(icase == 1):
        ax = 0 ; bx = 1 
        ay = 0 ; by = 1 
    elif(icase == 2):
        ax = 0 ; bx = 1 
        ay = 0 ; by = 1 
    elif(icase == 3):
        ax = -1 ; bx = 1
        ay = -1 ; by = 1
    elif(icase == 4):
        ax = -1 ; bx = 1
        ay = -1 ; by = 1
    elif(icase == 5):
        ax = -1 ; bx = 1
        ay = -1 ; by = 1
        
    print("==========================================")
    print("Problem: Poisson")
    print("Domain: [{}, {}]".format(ax,bx)) 
    return ax, bx,ay, by


# Boundary conditions 
    
def bc_type(x_boundary,y_boundary):
    
    bound = array([x_boundary,y_boundary])
    
    sbound = ["Dirichlet", "Neumann"]
    nbound = [5,4]
    
    st = ""
    
    for i in range(2):
        
        if(nbound[i] in bound):
            st += sbound[i] + " & "
            
    print("Boundary conditions: ",st)
    
    print("==========================================\n") 

In [ ]:
order = array([2])        # polynomial order
N_element = array([8])

integration_type = 1      # % = 1 is inexact and = 2 is exact
iplot = False             # plot the solution
icase = 3                 # select icase: 1,2,3

convergence_studies = True
iplot = False

# Boundary type: 4 = Neumann, 5 = Dirichlet     
x_boundary = [4,4]    # Bottom and Top (x = -1 and x = +1)
y_boundary = [4,4]    # Left and Right (y = -1 and x = +1)

# Domain and diffusion coefficient

ax,bx,ay, by = domain(icase)
bc_type(x_boundary,y_boundary)

len_el = len(N_element)
len_pol = len(order)
l2e_norm = zeros((len_pol, len_el))
max_norm = zeros((len_pol, len_el))

Nv = N_element
    
for iN,N in enumerate(order):
    
    N = order[iN]
    if (integration_type == 1):
        Q = N
    elif (integration_type == 2):
        Q = N+1

    for e, nel in enumerate(Nv):
            
        Nelx = nel; Nely = nel
        Nx = Nelx*N+1
        Ny = Nely*N+1
        Np = Nx*Ny
        Ne = Nelx*Nely
        Nbound = 2*Nx + 2*(Ny-2)
        Nside = 2*Ne + Nelx + Nely
        
        tic = perf_counter()
        
    
        qe, q,coord,intma = poisson_solver(N,Q,Ne, Np, ax, bx,ay,by, Nelx, Nely, Nx, Ny, \
                                    Nbound,Nside,icase,x_boundary,y_boundary)
        
        
        top = sum((q - qe)**2)
        bot = sum(qe**2)

        e2 = sqrt(top/bot)
        
        print("\tl2_norm = {:.4e}".format(e2))
        
        l2e_norm[iN,e] = e2
        
        
        

In [ ]:
if(iplot):
    xmin = min(coord[:,0])
    xmax = max(coord[:,0])
    ymin = min(coord[:,1])
    ymax = max(coord[:,1])
    xe = coord[:,0]
    ye = coord[:,1]
    nx = 200
    ny = 200
    dx = (xmax-xmin)/nx
    dy = (ymax-ymin)/ny
    x1 = arange(xmin,xmax+dx,dx)
    y1 = arange(ymin,ymax+dy,dy)
    xi,yi = meshgrid(x1,y1)


    q_2d = griddata((xe,ye),q,(xi,yi), method='cubic')

    qe_2d = griddata((xe,ye),qe,(xi,yi), method='cubic')

    ## First off, let's change the font size for all of our plots to be more legible
    plt.rcParams.update({'font.size': 14})

    fig = plt.figure(1,figsize=(15,12))

    fx = fig.add_subplot(121, projection='3d')
    surf = fx.plot_surface(xi,yi,q_2d,rstride = 1, cstride = 1,cmap=cm.coolwarm,antialiased=False)
    fig.colorbar(surf, shrink = 0.2)

    plt.title("Numerical solution")
    plt.xlabel("x")
    plt.ylabel("y")

    fx = fig.add_subplot(122, projection='3d')
    surf = fx.plot_surface(xi,yi,qe_2d,rstride = 1, cstride = 1,cmap=cm.coolwarm,antialiased=False)
    fig.colorbar(surf,shrink = 0.2)
    plt.title("Exact solution")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.show()


In [ ]:
order = array([1, 2, 3, 4, 5])        # polynomial order
Nv = array([8,16,24,32])

# Create a data type for storing results
dt_data = dtype([('N',int),('Np','int'),('1-norm','d'),('2-norm','d'),('inf-norm','d')])  

def poisson_simulation(file,iN, N,integration_type,Nv,x_boundary,y_boundary,ax,bx,ay, by):
    
    
    Q = N

    for e, nel in enumerate(Nv):
      
        
        Np = nel*N + 1

        Nelx = nel; Nely = nel
        Nx = Nelx*N+1
        Ny = Nely*N+1
        Np = Nx*Ny
        Ne = Nelx*Nely
        Nbound = 2*Nx + 2*(Ny-2)
        Nside = 2*Ne + Nelx + Nely
        
        tic = perf_counter()
        
    
        qe, q,coord,intma = poisson_solver(N,Q,Ne, Np, ax, bx,ay,by, Nelx, Nely, Nx, Ny, \
                                    Nbound,Nside,icase,x_boundary,y_boundary)
        
        
        error = abs(q-qe)
        top = sum(error**2)
        bot = sum(qe**2)

        
        
        #print("\tl2_norm = {:.4e}".format(e2))

        e1 = sum(error)
        e3 = max(error)
        e2 = sqrt(top/bot)

        t = array((nel,Np ,e1,e2,e3),dtype=dt_data)
        file.write(t)



# Poisson Simulation

if(convergence_studies):
    output_file = 'Neumann_py/poisson_44.dat'

    integType = [1]


    # Simulation. Loop over integration type, and then loop over polynomial order. 
    file = open(output_file,"wb")
    file.write(array([len(Nv)]))

    for integration_type in integType:

        for iN,N in enumerate(order):

            # Run over range of N = 8, 16, 32,64,128,256,...
            file.write(array((N,integration_type)))

            print("order = {:d}; Integration_type = {:d}".format(N,integration_type))
            poisson_simulation(file,iN, N,integration_type,Nv,x_boundary,y_boundary,ax,bx,ay, by)
            #print("")

    file.close()
    print("End")
